In [ ]:
! pip install geopandas

In [ ]:
import s3fs
import mlflow
import re
import numpy as np
from astrovision.plot import plot_images

import geopandas as gpd
import pandas as pd

import yaml
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from pyproj import Transformer
from shapely.geometry import Polygon

from astrovision.data import SatelliteImage, SegmentationLabeledSatelliteImage

Récupération d'un modèle et test

In [ ]:
%env MLFLOW_TRACKING_URI=https://projet-slums-detection-128833.user.lab.sspcloud.fr
%env MLFLOW_S3_ENDPOINT_URL=https://minio.lab.sspcloud.fr

# aller dans mlflow cliquer sur un modl et cliquer sur register pour créer ces variables
model_name = "test"
model_version = "13"

model_mlflow = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")
n_bands = int(mlflow.get_run(model_mlflow.metadata.run_id).data.params["n_bands"])

In [ ]:
# Import normalization metrics
params = yaml.safe_load(
    mlflow.artifacts.load_text(
        f"{mlflow.get_run(model_mlflow.metadata.run_id).info.artifact_uri}/model/code/metrics-normalization.yaml"
    )
)
normalization_mean, normalization_std = params["mean"], params["std"]
normalization_mean, normalization_std = (
    normalization_mean[:n_bands],
    normalization_std[:n_bands],
)

# Load an image
si = SatelliteImage.from_raster(
    file_path="/vsis3/projet-slums-detection/golden-test/patchs/segmentation/PLEIADES/MAYOTTE_CLEAN/2022/250/ORT_976_2022_0524_8587_U38S_8Bits_0005.jp2",
    dep=None,
    date=None,
    n_bands=n_bands,
)

# reproduce transform
transform = A.Compose(
    [
        A.Normalize(
            max_pixel_value=255.0,
            mean=normalization_mean,
            std=normalization_std,
        ),
        ToTensorV2(),
    ]
)

# normalize the image
# normalized_si = transform(image=np.transpose(si.array, [1, 2, 0]))["image"].unsqueeze(dim=0).numpy()
normalized_si = transform(image=np.transpose(si.array, [1, 2, 0]))["image"].unsqueeze(dim=0)

# predict the mask
# mask = (torch.tensor(model.predict(normalized_si)).softmax(dim=1)[:,1,:,:] > 0.5).numpy()[0,:,:]
# mask = (torch.tensor(model.predict(normalized_si)).sigmoid() > 0.5).numpy()
mask = model_mlflow.predict(normalized_si).sigmoid() > 0.5

lsi = SegmentationLabeledSatelliteImage(si, mask)

plot = lsi.plot(bands_indices=[0, 1, 2])

### Chargement des îlots

Ya plus les IOU ?
stocvkage moyenne variances dans les artifact MLFlow
Ici je vais charger une table de contour géométriique d'ilot et créer une fonction qui à partir d'un contour polygonal d'îlot  et d'un modèle :

1) récupère les images s'intersectant avec l'îlot (une année donnée) via leur nom (cooronnées du coin incluse dans le nom)
2) un modèle vient faire des prédictions 01 ou en proba sur les images récupérées
3) On somme les probabilités ou les 1 contenus dans l'ilôt (partie la plus subtile ? peut on cropper un raster ? je l'espère)
4) on peut donc attribuer à l'îlot uner surface batie prédite par le modèle
5) se permettre une représentation cartographique en représentant sur un même graphe le fond de carte donné par les patchs qui entour e complètement l'ilot (zone rectangulaire), les prédictions du modèle, et lîlot en question (ou bien d'une prédiction de réseau de neurones en probabilités) somme les1 du masque dans le contour de l'ilot

Je travaille à partir du jeu de test d'images de Mayotte dans un premeir temps ppour avoir des masques prédits hors modèle et des ilots martiniques

Petites remarques comme ça : Quand on est en prod on veut toutes les images du coup dans l'idée plutôt chopper les grosses images qui s'intersectent dans l'ilot, les spliter faire les prédictions et ne garder que les points dans l'ilot : moralité On repart des grosses images m^me si l'intersection est moins fine 
Problème d'EPSG

PB : l'installation de requirements marche pas

 - ok dans  un premier temps je vais prendre l'le nom de l'image  récupérer les coordonnées
 - les transformer en point
 - et trouver quel ilot intersecte l'image


mc cp --recursive  s3/projet-slums-detection/ilots/ ilots

In [ ]:
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://" + "minio.lab.sspcloud.fr"})
fs.download(rpath="projet-slums-detection/ilots", lpath="ilots", recursive=True)

In [ ]:
# Load a shapefile
fp = "ilots/ilots_976.shp"
ilots_mayotte = gpd.read_file(fp)
ilots_mayotte.head()

Je vais sélectionner les images qui tombent dans un ilot (ie où il y a intersection) et voir si je ne peux pas mettre 0 partou à l'extérieur de l'ilot

In [ ]:
bucket_name = "projet-slums-detection"
directory_path = "data-raw/PLEIADES/MAYOTTE/2022/"

full_path = f"{bucket_name}/{directory_path}"
files = fs.ls(full_path)

print(files[:6])

In [ ]:
# Function to extract coordinates from filename, adjusted for meters
def extract_coordinates(filename):
    matches = re.findall(r"\d{4}", filename)
    if len(matches) >= 2:
        return int(matches[-2]) * 1000, int(matches[-1]) * 1000
    return None, None


# Initialize the transformer from EPSG:4471 to WGS84
transformer = Transformer.from_crs("EPSG:4471", "EPSG:4326")

# Define the length of the square's side (in meters)
side_length = 1000  # Adjust as needed

# Process filenames and transform coordinates to create squares
data = []
for filename in files:
    x, y = extract_coordinates(filename)

    if x and y:
        # Define the square's corners in EPSG:4471
        top_left = (x, y)
        top_right = (x + side_length, y)
        bottom_right = (x + side_length, y - side_length)
        bottom_left = (x, y - side_length)

        # Create a polygon from these corners
        square = Polygon([top_left, top_right, bottom_right, bottom_left])

        # Transform each corner to latitude and longitude
        transformed_corners = [
            (lat, lon)
            for lon, lat in [transformer.transform(*corner) for corner in square.exterior.coords]
        ]

        # Create a new polygon with the transformed coordinates
        square_lat_lon = Polygon(transformed_corners)

        # Store the filename and the polygon
        data.append({"filename": filename, "geometry": square_lat_lon})

# Create a DataFrame
df = pd.DataFrame(data)

filename_to_polygon = gpd.GeoDataFrame(df, geometry="geometry", crs=4326)
filename_to_polygon.crs = "EPSG:4326"
# This DataFrame now contains the filenames and the corresponding square polygons in latitude and longitude
filename_to_polygon.head()

In [ ]:
intersection_ilot_filename = gpd.sjoin(
    ilots_mayotte, filename_to_polygon, how="inner", predicate="intersects"
)

intersection_ilot_filename.groupby("code_ilot").size().reset_index(name="count").sort_values(
    by="count", ascending=False
)

TO DO : représenter les ilots et les images sur un meme graph
Travail sur l'ilot 802 qui n'intersecte qu'une imagee :
1) récupération du polygon carré et de l'ilot
2) rasteriser l'intersection



In [ ]:
filename_ilot = intersection_ilot_filename.loc[
    intersection_ilot_filename["code_ilot"] == "0802"
].filename

polygon_ilot = intersection_ilot_filename.loc[
    intersection_ilot_filename["code_ilot"] == "0802"
].geometry.iloc[0]

polygon_ilot

In [ ]:
filename_to_polygon

In [ ]:
print(type(filename_ilot.iloc[0]))
filename_ilot.iloc[0]

In [ ]:
intersection_ilot_filename
polygon_image = filename_to_polygon.loc[
    filename_to_polygon["filename"] == filename_ilot.iloc[0]
].geometry.iloc[0]

polygon_image

In [ ]:
intersection_ilot_image_polygon = polygon_image.intersection(polygon_ilot)
intersection_ilot_image_polygon

In [ ]:
from shapely.geometry import MultiPolygon

multipolygone_union = MultiPolygon([polygon_image, intersection_ilot_image_polygon])
multipolygone_union

### Plus qu'à rasteriser tout ça

In [ ]:
import rasterio
from rasterio.features import rasterize

# rasterio.transform.from_bounds
x_min, y_min, x_max, y_max = polygon_image.bounds
transform = rasterio.transform.from_bounds(x_min, y_min, x_max, y_max, 2000, 2000)

raster_complet = np.zeros((2000, 2000), dtype=np.uint8)

valeur_polygone = 1

# Rasteriser le polygone actuel avec une valeur unique
raster_polygone = rasterize(
    [(intersection_ilot_image_polygon, valeur_polygone)],
    out_shape=(2000, 2000),
    transform=transform,
)

# Mettre à jour le raster complet avec les valeurs du polygone actuel
# Note: cela écrasera les valeurs dans les zones de chevauchement
raster_complet[raster_polygone > 0] = valeur_polygone

In [ ]:
import matplotlib.pyplot as plt
# Afficher l'image satellite
# plt.imshow(image_satellite, cmap='gray')  # Utilisez cmap='gray' pour une image en niveaux de gris ou omettez pour une image RGB

# Superposer le raster
# Utilisez 'alpha' pour régler la transparence de la superposition du raster
plt.imshow(
    raster_complet, cmap="jet", alpha=0.5
)  # 'jet' est un exemple de colormap, ajustez selon le résultat désiré

# Optionnel : Ajouter des titres, légendes, etc.
plt.title("Image Satellite avec Polygone Rasterisé Superposé")
plt.xlabel("Coordonnée X")
plt.ylabel("Coordonnée Y")

# Afficher le graphique
plt.show()

Objectif : superposer la mosaique d'imlage extraite et le cntour d'ilot
Superposer les prédictions du modèle
comment cropper un raster avec un polygone ?

#filename_ilot , créer la liste de si et faire la mosaique avec astrovision

In [ ]:
list_si = [
    SatelliteImage.from_raster(
        file_path="/vsis3/" + fp,
        dep=None,
        date=None,
        n_bands=3,
    )
    for fp in filename_ilot
]

In [ ]:
list_si[0].plot([0, 1, 2])

In [ ]:
plot_images(list_si, [0, 1, 2])

## Test API

In [ ]:
%env MLFLOW_TRACKING_URI=https://projet-slums-detection-128833.user.lab.sspcloud.fr
%env MLFLOW_S3_ENDPOINT_URL=https://minio.lab.sspcloud.fr
!AWS_ACCESS_KEY_ID=`vault kv get -field=ACCESS_KEY_ID onyxia-kv/projet-slums-detection/s3` && export AWS_ACCESS_KEY_ID
!AWS_SECRET_ACCESS_KEY=`vault kv get -field=SECRET_ACCESS_KEY onyxia-kv/projet-slums-detection/s3` && export AWS_SECRET_ACCESS_KEY
!unset AWS_SESSION_TOKEN

In [ ]:
import requests
import numpy as np
from astrovision.data import SatelliteImage, SegmentationLabeledSatelliteImage

image_path = "projet-slums-detection/golden-test/patchs/segmentation/PLEIADES/MAYOTTE_CLEAN/2022/250/ORT_976_2022_0524_8587_U38S_8Bits_0005.jp2"
response = requests.get(
    "https://satellite-images-inference.lab.sspcloud.fr/predict", params={"image": image_path}
)

lsi = SegmentationLabeledSatelliteImage(
    SatelliteImage.from_raster(
        file_path=f"/vsis3/{image_path}",
        dep=None,
        date=None,
        n_bands=3,
    ),
    np.array(response.json()["mask"]),
)
plot = lsi.plot(bands_indices=[0, 1, 2])